## 크롤링 🧐

## 1. 프로젝트 생성

In [ ]:
!pip install fake_useragent scrapy-fake-useragent

In [1]:
import scrapy, requests, re
from scrapy.http import TextResponse
from selenium import webdriver

In [2]:
!rm -rf musinsa
!scrapy startproject musinsa

New Scrapy project 'musinsa', using template directory '/home/ubuntu/.pyenv/versions/3.6.9/envs/python3/lib/python3.6/site-packages/scrapy/templates/project', created in:
    /home/ubuntu/python3/notebook/project/crawling-repo-3/musinsa

You can start your first spider with:
    cd musinsa
    scrapy genspider example example.com


In [2]:
!ls

README.md  mss-crawler-top-Copy1.ipynb	mss-crawler-top.ipynb  musinsa	run.sh


In [2]:
!tree musinsa/

musinsa/
├── musinsa
│   ├── __init__.py
│   ├── items.py
│   ├── middlewares.py
│   ├── pipelines.py
│   ├── settings.py
│   └── spiders
│       └── __init__.py
└── scrapy.cfg

2 directories, 7 files


## 2. xpath 확인

### link

In [15]:
req = requests.get("https://search.musinsa.com/category/001003")
response = TextResponse(req.url, body=req.text, encoding="utf-8")

In [16]:
links = response.xpath('//*[@id="searchList"]/li/div[contains(@class,"li_inner")]\
/div[2]/p[2]/a/@href').extract()
len(links), links[0]

(83, 'https://store.musinsa.com/app/goods/855195')

In [16]:
# 페이지 당 개수는 최대 90개

### info

In [28]:
link = "https://store.musinsa.com/app/goods/859956"
print(link)

https://store.musinsa.com/app/goods/859956


In [29]:
req = requests.get(link)
response = TextResponse(req.url, body=req.text, encoding="utf-8")

In [30]:
kw = response.xpath('//*[@id="product_order_info"]/div[1]/ul/li[contains(@class, "article-tag-list")]/p/a/text()').extract()
kw = list(map(lambda s : s.replace("#",''), kw))
kw

['남방', '셔츠', '오버사이즈셔츠', '체크', '체크무늬셔츠', '체크셔츠']

In [31]:
title = response.xpath('/html/head/title/text()')[0].extract().split(' -')[0]
title = title.split(') ')[1]

brand = response.xpath('//*[@id="product_order_info"]/div[1]/ul/li[1]/p[2]/strong/a/text()')[0].extract()

try:
    o_price = response.xpath('//*[@id="goods_price"]/del/text()')[0].extract().strip()
except:
    o_price = response.xpath('//*[@id="goods_price"]/text()')[0].extract().strip()

try:
    s_price = response.xpath('//*[@id="sale_price"]/text()')[0].extract().strip()
except:
    s_price = o_price

kw = response.xpath('//*[@id="product_order_info"]/div[1]/ul/li[contains(@class, "article-tag-list")]/p/a/text()').extract()
kw = list(map(lambda s : s.replace("#",''), kw))

img_link = "https:" + response.xpath('//*[@id="detail_bigimg"]/div[1]/img/@src')[0].extract()

link = response.url

item_id = link.split('/')[-1]

size_kind = response.xpath('//*[@id="size_table"]/tbody/tr/th/text()').extract()
size_details = {}

for idx in range(len(size_kind)-1):
    sd = response.xpath(f'//*[@id="size_table"]/tbody/tr[{idx+3}]/*/text()').extract()
    size_details[sd[0]] = sd[1:]
    
    
    
# selenium 시작


options = webdriver.ChromeOptions()
options.add_argument("headless")
driver = webdriver.Chrome(options=options)        
driver.get(link)

try:
    most_age = driver.find_elements_by_xpath('//*[@id="graph_summary_area"]/strong[1]')[0].text
except:
    most_age = 0
try:
    most_sex = driver.find_elements_by_xpath('//*[@id="graph_summary_area"]/span[1]')[0].text
except:
    most_sex = 0
    
option1_ls = driver.find_elements_by_xpath('//*[@id="option1"]/option[not(@value="")]')
option1 = [option.get_attribute("value") for option in option1_ls]
driver.find_element_by_xpath('//*[@id="option1"]/option[not(@value="")]').click()

option2_ls = driver.find_elements_by_xpath('//*[@id="option2"]/option[not(@value="")]')
option2 = [option.get_attribute("value") for option in option2_ls]


for_size = ['90', '95', 'S', 'M', 'L', 'LARGE', '1', 'M_쭈리']
if any(x in for_size for x in option1):
    sizes = option1
    colors = option2
else:
    sizes = option2
    colors = option1

driver.quit()


# selenium 끝



# 결과 확인

print(
    f"title : {title}" "\n" f'brand : {brand}' "\n"
    f"o_price : {o_price}" "\n" f's_price : {s_price}' "\n"
    f"kw : {kw}" "\n" f'img_link : {img_link}' "\n"
    f"link : {link}" "\n" f'item_id : {item_id}' "\n"
    f"most_age : {most_age}" "\n" f'most_sex : {most_sex}' "\n"
    f"colors : {colors}" "\n" f'sizes : {sizes}' "\n" f'size_details : {size_details}' "\n"
)

title : 무드 체크 셔츠 S24 네이비
brand : DIAMOND LAYLA
o_price : 68,000
s_price : 43,000
kw : ['남방', '셔츠', '오버사이즈셔츠', '체크', '체크무늬셔츠', '체크셔츠']
img_link : https://image.msscdn.net/images/goods_img/20180917/859956/859956_4_500.jpg
link : https://store.musinsa.com/app/goods/859956
item_id : 859956
most_age : 24~28세
most_sex : 남성
colors : []
sizes : ['S', 'M', 'L', 'XL']
size_details : {'S': ['72.5', '54.5', '55', '56'], 'M': ['75', '56', '58', '58'], 'L': ['77.5', '57.5', '61', '60'], 'XL': ['80', '59', '64', '61']}



In [12]:
# driver.quit()

## 3. items.py

In [3]:
%%writefile musinsa/musinsa/items.py
import scrapy


class MusinsaItem(scrapy.Item):
    title = scrapy.Field()
    brand = scrapy.Field()
    o_price = scrapy.Field()
    s_price = scrapy.Field()
    kw = scrapy.Field()
    img_link = scrapy.Field()
    link = scrapy.Field()
    item_id = scrapy.Field()
    size_details = scrapy.Field()
    most_age = scrapy.Field()
    most_sex = scrapy.Field()
    colors = scrapy.Field()
    sizes = scrapy.Field()

Overwriting musinsa/musinsa/items.py


## 4. spider.py
### 한 페이지 (최대 90개)

In [4]:
%%writefile musinsa/musinsa/spiders/spider.py

import scrapy
from musinsa.items import MusinsaItem
from fake_useragent import UserAgent


class Spider(scrapy.Spider):
    name = "Musinsa"
    allow_domain = ["musinsa.com"]
    start_urls = ["https://search.musinsa.com/category/001002"]
    custom_settings = {
        'CONCURRENT_REQUESTS': 10,
#         'DOWNLOAD_DELAY': 2.0,
        'AUTOTHROTTLE_ENABLED': True,
        "DOWNLOADER_MIDDLEWARES": {
            "scrapy.downloadermiddlewares.useragent.UserAgentMiddleware": None,
            'scrapy.downloadermiddlewares.retry.RetryMiddleware': None,
            "scrapy_fake_useragent.middleware.RandomUserAgentMiddleware": 400,
            'scrapy_fake_useragent.middleware.RetryUserAgentMiddleware': 401,
        }
    }
    
    def parse(self, response):
        links = response.xpath('//*[@id="searchList"]/li/div[contains(@class,"li_inner")]\
/div[2]/p[2]/a/@href').extract()
        for link in links:
            yield scrapy.Request(link, callback=self.parse_content)
            
    def parse_content(self, response):
        item = MusinsaItem()
        title = response.xpath('/html/head/title/text()')[0].extract().split(' -')[0]
        item["title"] = title.split(') ')[1]
        item["brand"] = response.xpath('//*[@id="product_order_info"]/div[1]/ul/li[1]/p[2]/strong/a/text()')[0].extract()
        try:
            item["o_price"] = response.xpath('//*[@id="goods_price"]/del/text()')[0].extract().strip()
        except:
            item["o_price"] = response.xpath('//*[@id="goods_price"]/text()')[0].extract().strip()
        try:
            item["s_price"] = response.xpath('//*[@id="sale_price"]/text()')[0].extract().strip()
        except:
            item["s_price"] = item["o_price"]
        kw = response.xpath('//*[@id="product_order_info"]/div[1]/ul/li[contains(@class, "article-tag-list")]/p/a/text()').extract()
        item["kw"] = list(map(lambda s : s.replace("#",''), kw))
        item["img_link"] = "https:" + response.xpath('//*[@id="detail_bigimg"]/div[1]/img/@src')[0].extract()
        item["link"] = response.url
        item["item_id"] = item["link"].split('/')[-1]
        size_kind = response.xpath('//*[@id="size_table"]/tbody/tr/th/text()').extract()
        item["size_details"] = {}
        for idx in range(len(size_kind)-1):
            sd = response.xpath(f'//*[@id="size_table"]/tbody/tr[{idx+3}]/*/text()').extract()
            item["size_details"][sd[0]] = sd[1:]

        yield item

Writing musinsa/musinsa/spiders/spider.py


## 4'. spider.py
### selenium part

In [11]:
%%writefile musinsa/musinsa/spiders/spider.py

import scrapy
from musinsa.items import MusinsaItem
from selenium import webdriver
from fake_useragent import UserAgent


class Spider(scrapy.Spider):
    name = "Musinsa"
    allow_domain = ["musinsa.com"]
    start_urls = ["https://search.musinsa.com/category/001003"]
    custom_settings = {
        'CONCURRENT_REQUESTS': 3,
        'DOWNLOAD_DELAY': 6.0,
        'AUTOTHROTTLE_ENABLED': True,
        "DOWNLOADER_MIDDLEWARES": {
            "scrapy.downloadermiddlewares.useragent.UserAgentMiddleware": None,
            "scrapy_fake_useragent.middleware.RandomUserAgentMiddleware": 400,
        }
    }
    
    def parse(self, response):
        links = response.xpath('//*[@id="searchList"]/li/div[contains(@class,"li_inner")]\
/div[2]/p[2]/a/@href').extract()
        for link in links:
            yield scrapy.Request(link, callback=self.parse_content)
            
    def parse_content(self, response):     
        item = MusinsaItem()
        options = webdriver.ChromeOptions()
        options.add_argument("headless")
        options.add_argument("user-agent={}".format(UserAgent().Chrome))
        driver = webdriver.Chrome(options=options)        
        driver.get(response.url)       
        
        try:
            item["most_age"] = driver.find_elements_by_xpath('//*[@id="graph_summary_area"]/strong[1]')[0].text
        except:
            item["most_age"] = 0
        try:
            item["most_sex"] = driver.find_elements_by_xpath('//*[@id="graph_summary_area"]/span[1]')[0].text
        except:
            item["most_sex"] = 0

        option1_ls = driver.find_elements_by_xpath('//*[@id="option1"]/option[not(@value="")]')
        option1 = [option.get_attribute("value") for option in option1_ls]
        driver.find_element_by_xpath('//*[@id="option1"]/option[not(@value="")]').click()
        option2_ls = driver.find_elements_by_xpath('//*[@id="option2"]/option[not(@value="")]')
        option2 = [option.get_attribute("value") for option in option2_ls]

        for_size = ['90', '95', 'S', 'M', 'L', 'LARGE', '1', 'M_쭈리']
        if any(x in for_size for x in option1):
            item["sizes"] = option1
            item["colors"] = option2
        else:
            item["sizes"] = option2
            item["colors"] = option1

        driver.quit()
        
        item["item_id"] = response.url.split('/')[-1]
        
        yield item

Overwriting musinsa/musinsa/spiders/spider.py


## 5. run scrapy

In [5]:
%%writefile run.sh
cd musinsa
scrapy crawl Musinsa -o ./datas/top.csv

Overwriting run.sh


### settings.py 변경

In [6]:
!head -n 20 musinsa/musinsa/settings.py | tail -n 2

# Obey robots.txt rules
ROBOTSTXT_OBEY = True


In [7]:
!sed -i 's/ROBOTSTXT_OBEY = True/ROBOTSTXT_OBEY = False/' musinsa/musinsa/settings.py

In [8]:
!head -n 20 musinsa/musinsa/settings.py | tail -n 2

# Obey robots.txt rules
ROBOTSTXT_OBEY = False


In [9]:
%%time
!/bin/bash run.sh

2021-02-26 20:59:31 [scrapy.utils.log] INFO: Scrapy 2.4.1 started (bot: musinsa)
2021-02-26 20:59:31 [scrapy.utils.log] INFO: Versions: lxml 4.6.2.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 20.3.0, Python 3.6.9 (default, Dec 15 2020, 04:39:26) - [GCC 7.5.0], pyOpenSSL 20.0.1 (OpenSSL 1.1.1i  8 Dec 2020), cryptography 3.3.1, Platform Linux-5.4.0-1037-aws-x86_64-with-debian-buster-sid
2021-02-26 20:59:31 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.epollreactor.EPollReactor
2021-02-26 20:59:31 [scrapy.crawler] INFO: Overridden settings:
{'AUTOTHROTTLE_ENABLED': True,
 'BOT_NAME': 'musinsa',
 'CONCURRENT_REQUESTS': 10,
 'NEWSPIDER_MODULE': 'musinsa.spiders',
 'SPIDER_MODULES': ['musinsa.spiders']}
2021-02-26 20:59:31 [scrapy.extensions.telnet] INFO: Telnet Password: 562758b7d3f361c4
2021-02-26 20:59:31 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensi

2021-02-26 20:59:43 [scrapy_fake_useragent.middleware] DEBUG: Loaded User-Agent provider: scrapy_fake_useragent.providers.FakeUserAgentProvider
2021-02-26 20:59:43 [scrapy_fake_useragent.middleware] INFO: Using '<class 'scrapy_fake_useragent.providers.FakeUserAgentProvider'>' as the User-Agent provider
2021-02-26 20:59:43 [scrapy_fake_useragent.middleware] DEBUG: Loaded User-Agent provider: scrapy_fake_useragent.providers.FakeUserAgentProvider
2021-02-26 20:59:43 [scrapy_fake_useragent.middleware] INFO: Using '<class 'scrapy_fake_useragent.providers.FakeUserAgentProvider'>' as the User-Agent provider
2021-02-26 20:59:43 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 'scrapy.downloadermiddlewares.downloadtimeout.DownloadTimeoutMiddleware',
 'scrapy.downloadermiddlewares.defaultheaders.DefaultHeadersMiddleware',
 'scrapy_fake_useragent.middleware.RandomUserAgentMiddleware',
 'scrapy_fake_useragent.middleware.RetryUs

2021-02-26 20:59:58 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://store.musinsa.com/app/goods/1008043> (referer: https://search.musinsa.com/category/001002)
2021-02-26 20:59:58 [scrapy.core.scraper] DEBUG: Scraped from <200 https://store.musinsa.com/app/goods/1008043>
{'brand': 'ANOBLIR',
 'img_link': 'https://image.msscdn.net/images/goods_img/20190408/1008043/1008043_2_500.jpg',
 'item_id': '1008043',
 'kw': ['스마트룩', '리넨', '오픈카라', '체크', '스트라이프', '오버핏', '실키', '린넨셔츠'],
 'link': 'https://store.musinsa.com/app/goods/1008043',
 'o_price': '60,000',
 's_price': '42,000',
 'size_details': {'L': ['75', '51', '56.5', '59.5'],
                  'M': ['73.5', '49', '54', '58.5'],
                  'S': ['72', '47', '51.5', '57.5'],
                  'XL': ['76.5', '53', '59', '60.5']},
 'title': '루즈핏 오픈카라 텐셀 린넨 셔츠 [네이비]'}
2021-02-26 20:59:58 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://store.musinsa.com/app/goods/1015136> (referer: https://search.musinsa.com/category/001002)
2

2021-02-26 21:00:01 [scrapy.core.scraper] DEBUG: Scraped from <200 https://store.musinsa.com/app/goods/876307>
{'brand': 'MUSINSA STANDARD',
 'img_link': 'https://image.msscdn.net/images/goods_img/20181010/876307/876307_2_500.jpg',
 'item_id': '876307',
 'kw': ['겨울', '골덴셔츠', '기본', '베이직', '코듀로이컬렉션', '긴팔셔츠'],
 'link': 'https://store.musinsa.com/app/goods/876307',
 'o_price': '28,900',
 's_price': '11,590',
 'size_details': {'L': ['75', '47', '55.5', '63'],
                  'M': ['73.5', '45.4', '53', '62'],
                  'XL': ['76.5', '48.6', '58', '64'],
                  'XXL': ['78', '50.2', '60.5', '65']},
 'title': '베이식 코듀로이 셔츠 [머스타드]'}
2021-02-26 21:00:01 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://store.musinsa.com/app/goods/983341> (referer: https://search.musinsa.com/category/001002)
2021-02-26 21:00:02 [scrapy.core.scraper] DEBUG: Scraped from <200 https://store.musinsa.com/app/goods/983341>
{'brand': 'MUSINSA STANDARD',
 'img_link': 'https://image.msscdn.net/i

2021-02-26 21:00:06 [scrapy.core.scraper] DEBUG: Scraped from <200 https://store.musinsa.com/app/goods/1333644>
{'brand': 'MUSINSA STANDARD',
 'img_link': 'https://image.msscdn.net/images/goods_img/20200304/1333644/1333644_1_500.jpg',
 'item_id': '1333644',
 'kw': ['기능성', '사계절', '수트', '스트레치', '와이셔츠', '정장', '긴팔셔츠'],
 'link': 'https://store.musinsa.com/app/goods/1333644',
 'o_price': '27,900',
 's_price': '27,900',
 'size_details': {'2XL': ['80', '50.2', '60', '65'],
                  'L': ['77', '47', '55', '63'],
                  'M': ['75.5', '45.4', '52.5', '62'],
                  'XL': ['78.5', '48.6', '57.5', '64']},
 'title': '와이드 스프레드 칼라 드레스 셔츠 [화이트]'}
2021-02-26 21:00:06 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://store.musinsa.com/app/goods/1695221> (referer: https://search.musinsa.com/category/001002)
2021-02-26 21:00:06 [scrapy.core.scraper] DEBUG: Scraped from <200 https://store.musinsa.com/app/goods/1695221>
{'brand': 'WHOAU',
 'img_link': 'https://image.msscdn

...

2021-02-26 21:00:12 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://store.musinsa.com/app/goods/483798> (referer: https://search.musinsa.com/category/001002)
2021-02-26 21:00:12 [scrapy.core.scraper] DEBUG: Scraped from <200 https://store.musinsa.com/app/goods/483798>
{'brand': 'PERSTEP',
 'img_link': 'https://image.msscdn.net/images/goods_img/20170215/483798/483798_4_500.jpg',
 'item_id': '483798',
 'kw': ['루즈핏', '루즈핏셔츠', '셔츠', '오버핏', '오버핏셔츠', '플레인', '플레인셔츠'],
 'link': 'https://store.musinsa.com/app/goods/483798',
 'o_price': '33,900',
 's_price': '33,900',
 'size_details': {},
 'title': '피그먼트 루즈핏 셔츠 진회색 SMLS4058'}
2021-02-26 21:00:12 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://store.musinsa.com/app/goods/1208509> (referer: https://search.musinsa.com/category/001002)
2021-02-26 21:00:12 [scrapy.core.scraper] DEBUG: Scraped from <200 https://store.musinsa.com/app/goods/1208509>
{'brand': 'AVAN',
 'img_link': 'https://image.msscdn.net/images/goods_img/20191031/1208509/

2021-02-26 21:00:15 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://store.musinsa.com/app/goods/876309> (referer: https://search.musinsa.com/category/001002)
2021-02-26 21:00:15 [scrapy.core.scraper] DEBUG: Scraped from <200 https://store.musinsa.com/app/goods/876309>
{'brand': 'MUSINSA STANDARD',
 'img_link': 'https://image.msscdn.net/images/goods_img/20181010/876309/876309_2_500.jpg',
 'item_id': '876309',
 'kw': ['겨울', '골덴셔츠', '기본', '베이직', '코듀로이컬렉션', '긴팔셔츠'],
 'link': 'https://store.musinsa.com/app/goods/876309',
 'o_price': '28,900',
 's_price': '11,590',
 'size_details': {'L': ['75', '47', '55.5', '63'],
                  'M': ['73.5', '45.4', '53', '62'],
                  'XL': ['76.5', '48.6', '58', '64'],
                  'XXL': ['78', '50.2', '60.5', '65']},
 'title': '베이식 코듀로이 셔츠 [핑크]'}
2021-02-26 21:00:15 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://store.musinsa.com/app/goods/994588> (referer: https://search.musinsa.com/category/001002)
2021-02-26 21:00:1

2021-02-26 21:00:20 [scrapy.core.scraper] DEBUG: Scraped from <200 https://store.musinsa.com/app/goods/1296430>
{'brand': 'DIAMOND LAYLA',
 'img_link': 'https://image.msscdn.net/images/goods_img/20200207/1296430/1296430_1_500.jpg',
 'item_id': '1296430',
 'kw': ['남자셔츠', '미니멀', '스트라이프', '스트라이프셔츠', '오버핏', '커플셔츠', '포켓셔츠'],
 'link': 'https://store.musinsa.com/app/goods/1296430',
 'o_price': '65,000',
 's_price': '43,000',
 'size_details': {'L': ['77.5', '57.5', '61', '58'],
                  'M': ['75', '56', '58', '57'],
                  'S': ['72.5', '54.5', '55', '56'],
                  'XL': ['80', '59', '64', '59']},
 'title': '로프 스트라이프 셔츠 S46 스카이 블루'}
2021-02-26 21:00:21 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://store.musinsa.com/app/goods/876311> (referer: https://search.musinsa.com/category/001002)
2021-02-26 21:00:21 [scrapy.core.scraper] DEBUG: Scraped from <200 https://store.musinsa.com/app/goods/876311>
{'brand': 'MUSINSA STANDARD',
 'img_link': 'https://image.ms

2021-02-26 21:00:24 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://store.musinsa.com/app/goods/1116155> (referer: https://search.musinsa.com/category/001002)
2021-02-26 21:00:24 [scrapy.core.scraper] DEBUG: Scraped from <200 https://store.musinsa.com/app/goods/1116155>
{'brand': 'DIAMOND LAYLA',
 'img_link': 'https://image.msscdn.net/images/goods_img/20190812/1116155/1116155_3_500.jpg',
 'item_id': '1116155',
 'kw': ['긴팔', '긴팔셔츠', '남방', '셔츠', '체크', '체크무늬셔츠', '체크셔츠'],
 'link': 'https://store.musinsa.com/app/goods/1116155',
 'o_price': '69,000',
 's_price': '43,000',
 'size_details': {'L': ['77.5', '57.5', '61', '58'],
                  'M': ['75', '56', '58', '57'],
                  'S': ['72.5', '54.5', '55', '56'],
                  'XL': ['80', '59', '64', '59']},
 'title': '헤링본 클래식 체크 셔츠 S41'}
2021-02-26 21:00:24 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://store.musinsa.com/app/goods/854337> (referer: https://search.musinsa.com/category/001002)
2021-02-26 21:00:2

2021-02-26 21:00:27 [scrapy.core.scraper] DEBUG: Scraped from <200 https://store.musinsa.com/app/goods/728000>
{'brand': 'MUSINSA STANDARD',
 'img_link': 'https://image.msscdn.net/images/goods_img/20180305/728000/728000_4_500.jpg',
 'item_id': '728000',
 'kw': ['기본', '레귤러핏', '베이직', '사계절', '옥스포드셔츠', '긴팔셔츠'],
 'link': 'https://store.musinsa.com/app/goods/728000',
 'o_price': '24,900',
 's_price': '24,900',
 'size_details': {'L': ['75', '46.5', '55', '63'],
                  'M': ['73.5', '44.9', '52.5', '62'],
                  'XL': ['76.5', '48.1', '57.5', '64'],
                  'XXL': ['77.5', '49.7', '60', '64.5'],
                  'XXXL': ['78.5', '51.3', '62.5', '65']},
 'title': '베이식 옥스포드 셔츠 [화이트]'}
2021-02-26 21:00:27 [scrapy.core.engine] INFO: Closing spider (finished)
2021-02-26 21:00:27 [scrapy.extensions.feedexport] INFO: Stored csv feed (88 items) in: ./datas/top.csv
2021-02-26 21:00:27 [scrapy.statscollectors] INFO: Dumping Scrapy stats:
{'downloader/request_bytes': 4489

In [10]:
pd.read_csv('./musinsa/datas/top.csv').tail()

,brand,colors,img_link,item_id,kw,link,most_age,most_sex,o_price,s_price,size_details,sizes,title
83,DIAMOND LAYLA,NaN,https://image.msscdn.net/images/goods_img/2018...,826451,"남방,셔츠,스트라이프,스트라이프셔츠,오버사이즈셔츠,체크셔츠",https://store.musinsa.com/app/goods/826451,NaN,NaN,"65,000","43,000","{'S': ['72.5', '54.5', '55', '56'], 'M': ['75'...",NaN,빈티지 블랙 도트 스트라이프 셔츠 S22
84,TAKEASY,NaN,https://image.msscdn.net/images/goods_img/2020...,1336290,"레이어드,오버셔츠,오버핏,옥스포드",https://store.musinsa.com/app/goods/1336290,NaN,NaN,"67,000","40,200","{'S': ['75', '58', '68', '56'], 'M': ['77', '6...",NaN,솔리드 옥스포드 오버셔츠(화이트)
85,MUSINSA STANDARD,NaN,https://image.msscdn.net/images/goods_img/2019...,1068742,"기본셔츠,드레스셔츠,수트,와이셔츠,정장,클래식,포멀,긴팔셔츠",https://store.musinsa.com/app/goods/1068742,NaN,NaN,"26,900","16,190","{'M': ['76.5', '48.4', '53.5', '61'], 'L': ['7...",NaN,베이식 포플린 셔츠 [화이트]
86,JHONNY WEST,NaN,https://image.msscdn.net/images/goods_img/2018...,890749,"셔츠,오버핏셔츠,체크,체크무늬셔츠,체크셔츠,타탄체크",https://store.musinsa.com/app/goods/890749,NaN,NaN,"33,000","29,700","{'48(95)': ['76', '52', '66', '63'], '50(100)'...",NaN,Wooltan Check Shirts (Cream)
87,MUSINSA STANDARD,NaN,https://image.msscdn.net/images/goods_img/2018...,728000,"기본,레귤러핏,베이직,사계절,옥스포드셔츠,긴팔셔츠",https://store.musinsa.com/app/goods/728000,NaN,NaN,"24,900","24,900","{'M': ['73.5', '44.9', '52.5', '62'], 'L': ['7...",NaN,베이식 옥스포드 셔츠 [화이트]


## 5'. run scrapy
### selenium part

In [12]:
%%writefile run.sh
cd musinsa
scrapy crawl Musinsa -o ./datas/top_s.csv

Overwriting run.sh


### selenium part

In [13]:
%%time
!/bin/bash run.sh

2021-02-26 21:02:09 [scrapy.utils.log] INFO: Scrapy 2.4.1 started (bot: musinsa)
2021-02-26 21:02:09 [scrapy.utils.log] INFO: Versions: lxml 4.6.2.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 20.3.0, Python 3.6.9 (default, Dec 15 2020, 04:39:26) - [GCC 7.5.0], pyOpenSSL 20.0.1 (OpenSSL 1.1.1i  8 Dec 2020), cryptography 3.3.1, Platform Linux-5.4.0-1037-aws-x86_64-with-debian-buster-sid
2021-02-26 21:02:09 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.epollreactor.EPollReactor
2021-02-26 21:02:09 [scrapy.crawler] INFO: Overridden settings:
{'AUTOTHROTTLE_ENABLED': True,
 'BOT_NAME': 'musinsa',
 'CONCURRENT_REQUESTS': 3,
 'DOWNLOAD_DELAY': 6.0,
 'NEWSPIDER_MODULE': 'musinsa.spiders',
 'SPIDER_MODULES': ['musinsa.spiders']}
2021-02-26 21:02:09 [scrapy.extensions.telnet] INFO: Telnet Password: 25a0e1f126811369
2021-02-26 21:02:09 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetCon

2021-02-26 21:02:16 [urllib3.connectionpool] DEBUG: http://127.0.0.1:47687 "POST /session/4290522614c83079e7fa3df33340514c/execute/sync HTTP/1.1" 200 13
2021-02-26 21:02:16 [selenium.webdriver.remote.remote_connection] DEBUG: Finished Request
2021-02-26 21:02:16 [selenium.webdriver.remote.remote_connection] DEBUG: POST http://127.0.0.1:47687/session/4290522614c83079e7fa3df33340514c/execute/sync {"script": "return (function(){return function(){var d=this;function f(a){return\"string\"==typeof a};function h(a,b){this.code=a;this.a=l[a]||m;this.message=b||\"\";a=this.a.replace(/((?:^|\\s+)[a-z])/g,function(a){return a.toUpperCase().replace(/^[\\s\\xa0]+/g,\"\")});b=a.length-5;if(0>b||a.indexOf(\"Error\",b)!=b)a+=\"Error\";this.name=a;a=Error(this.message);a.name=this.name;this.stack=a.stack||\"\"}\n(function(){var a=Error;function b(){}b.prototype=a.prototype;h.b=a.prototype;h.prototype=new b;h.prototype.constructor=h;h.a=function(b,c,g){for(var e=Array(arguments.length-2),k=2;k<arguments

2021-02-26 21:02:16 [urllib3.connectionpool] DEBUG: http://127.0.0.1:47687 "POST /session/4290522614c83079e7fa3df33340514c/execute/sync HTTP/1.1" 200 13
2021-02-26 21:02:16 [selenium.webdriver.remote.remote_connection] DEBUG: Finished Request
2021-02-26 21:02:16 [selenium.webdriver.remote.remote_connection] DEBUG: POST http://127.0.0.1:47687/session/4290522614c83079e7fa3df33340514c/element {"using": "xpath", "value": "//*[@id=\"option1\"]/option[not(@value=\"\")]"}
2021-02-26 21:02:17 [urllib3.connectionpool] DEBUG: http://127.0.0.1:47687 "POST /session/4290522614c83079e7fa3df33340514c/element HTTP/1.1" 200 88
2021-02-26 21:02:17 [selenium.webdriver.remote.remote_connection] DEBUG: Finished Request
2021-02-26 21:02:17 [selenium.webdriver.remote.remote_connection] DEBUG: POST http://127.0.0.1:47687/session/4290522614c83079e7fa3df33340514c/element/4c6904ea-1b77-4cfb-be85-218341634814/click {"id": "4c6904ea-1b77-4cfb-be85-218341634814"}
2021-02-26 21:02:17 [urllib3.connectionpool] DEBUG: 

2021-02-26 21:02:22 [urllib3.connectionpool] DEBUG: http://127.0.0.1:55865 "POST /session/4b78100ca8a414df3b325269f915dff9/element/6e2e9d1e-98ba-4502-85b4-103ca738a6de/click HTTP/1.1" 200 14
2021-02-26 21:02:22 [selenium.webdriver.remote.remote_connection] DEBUG: Finished Request
2021-02-26 21:02:22 [selenium.webdriver.remote.remote_connection] DEBUG: POST http://127.0.0.1:55865/session/4b78100ca8a414df3b325269f915dff9/elements {"using": "xpath", "value": "//*[@id=\"option2\"]/option[not(@value=\"\")]"}
2021-02-26 21:02:22 [urllib3.connectionpool] DEBUG: http://127.0.0.1:55865 "POST /session/4b78100ca8a414df3b325269f915dff9/elements HTTP/1.1" 200 327
2021-02-26 21:02:22 [selenium.webdriver.remote.remote_connection] DEBUG: Finished Request
2021-02-26 21:02:22 [selenium.webdriver.remote.remote_connection] DEBUG: POST http://127.0.0.1:55865/session/4b78100ca8a414df3b325269f915dff9/execute/sync {"script": "return (function(){return function(){var d=this;function f(a){return\"string\"==type

2021-02-26 21:02:22 [urllib3.connectionpool] DEBUG: http://127.0.0.1:55865 "POST /session/4b78100ca8a414df3b325269f915dff9/execute/sync HTTP/1.1" 200 15
2021-02-26 21:02:22 [selenium.webdriver.remote.remote_connection] DEBUG: Finished Request
2021-02-26 21:02:22 [selenium.webdriver.remote.remote_connection] DEBUG: POST http://127.0.0.1:55865/session/4b78100ca8a414df3b325269f915dff9/execute/sync {"script": "return (function(){return function(){var d=this;function f(a){return\"string\"==typeof a};function h(a,b){this.code=a;this.a=l[a]||m;this.message=b||\"\";a=this.a.replace(/((?:^|\\s+)[a-z])/g,function(a){return a.toUpperCase().replace(/^[\\s\\xa0]+/g,\"\")});b=a.length-5;if(0>b||a.indexOf(\"Error\",b)!=b)a+=\"Error\";this.name=a;a=Error(this.message);a.name=this.name;this.stack=a.stack||\"\"}\n(function(){var a=Error;function b(){}b.prototype=a.prototype;h.b=a.prototype;h.prototype=new b;h.prototype.constructor=h;h.a=function(b,c,g){for(var e=Array(arguments.length-2),k=2;k<arguments

...

2021-02-26 21:13:28 [urllib3.connectionpool] DEBUG: http://127.0.0.1:52929 "POST /session/dd7dc6c84bdd670a1d7f864762029cd7/execute/sync HTTP/1.1" 200 13
2021-02-26 21:13:28 [selenium.webdriver.remote.remote_connection] DEBUG: Finished Request
2021-02-26 21:13:28 [selenium.webdriver.remote.remote_connection] DEBUG: POST http://127.0.0.1:52929/session/dd7dc6c84bdd670a1d7f864762029cd7/execute/sync {"script": "return (function(){return function(){var d=this;function f(a){return\"string\"==typeof a};function h(a,b){this.code=a;this.a=l[a]||m;this.message=b||\"\";a=this.a.replace(/((?:^|\\s+)[a-z])/g,function(a){return a.toUpperCase().replace(/^[\\s\\xa0]+/g,\"\")});b=a.length-5;if(0>b||a.indexOf(\"Error\",b)!=b)a+=\"Error\";this.name=a;a=Error(this.message);a.name=this.name;this.stack=a.stack||\"\"}\n(function(){var a=Error;function b(){}b.prototype=a.prototype;h.b=a.prototype;h.prototype=new b;h.prototype.constructor=h;h.a=function(b,c,g){for(var e=Array(arguments.length-2),k=2;k<arguments

2021-02-26 21:13:28 [urllib3.connectionpool] DEBUG: http://127.0.0.1:52929 "POST /session/dd7dc6c84bdd670a1d7f864762029cd7/execute/sync HTTP/1.1" 200 15
2021-02-26 21:13:28 [selenium.webdriver.remote.remote_connection] DEBUG: Finished Request
2021-02-26 21:13:28 [selenium.webdriver.remote.remote_connection] DEBUG: POST http://127.0.0.1:52929/session/dd7dc6c84bdd670a1d7f864762029cd7/element {"using": "xpath", "value": "//*[@id=\"option1\"]/option[not(@value=\"\")]"}
2021-02-26 21:13:28 [urllib3.connectionpool] DEBUG: http://127.0.0.1:52929 "POST /session/dd7dc6c84bdd670a1d7f864762029cd7/element HTTP/1.1" 200 88
2021-02-26 21:13:28 [selenium.webdriver.remote.remote_connection] DEBUG: Finished Request
2021-02-26 21:13:28 [selenium.webdriver.remote.remote_connection] DEBUG: POST http://127.0.0.1:52929/session/dd7dc6c84bdd670a1d7f864762029cd7/element/96bbb691-4a4e-437c-911b-f21b1a21d7d1/click {"id": "96bbb691-4a4e-437c-911b-f21b1a21d7d1"}
2021-02-26 21:13:28 [urllib3.connectionpool] DEBUG: 

2021-02-26 21:13:36 [urllib3.connectionpool] DEBUG: http://127.0.0.1:41779 "POST /session/1bc17f49757f2ac2366f3e111feec946/execute/sync HTTP/1.1" 200 13
2021-02-26 21:13:36 [selenium.webdriver.remote.remote_connection] DEBUG: Finished Request
2021-02-26 21:13:36 [selenium.webdriver.remote.remote_connection] DEBUG: POST http://127.0.0.1:41779/session/1bc17f49757f2ac2366f3e111feec946/execute/sync {"script": "return (function(){return function(){var d=this;function f(a){return\"string\"==typeof a};function h(a,b){this.code=a;this.a=l[a]||m;this.message=b||\"\";a=this.a.replace(/((?:^|\\s+)[a-z])/g,function(a){return a.toUpperCase().replace(/^[\\s\\xa0]+/g,\"\")});b=a.length-5;if(0>b||a.indexOf(\"Error\",b)!=b)a+=\"Error\";this.name=a;a=Error(this.message);a.name=this.name;this.stack=a.stack||\"\"}\n(function(){var a=Error;function b(){}b.prototype=a.prototype;h.b=a.prototype;h.prototype=new b;h.prototype.constructor=h;h.a=function(b,c,g){for(var e=Array(arguments.length-2),k=2;k<arguments

2021-02-26 21:13:37 [urllib3.connectionpool] DEBUG: http://127.0.0.1:41779 "POST /session/1bc17f49757f2ac2366f3e111feec946/execute/sync HTTP/1.1" 200 13
2021-02-26 21:13:37 [selenium.webdriver.remote.remote_connection] DEBUG: Finished Request
2021-02-26 21:13:37 [selenium.webdriver.remote.remote_connection] DEBUG: POST http://127.0.0.1:41779/session/1bc17f49757f2ac2366f3e111feec946/execute/sync {"script": "return (function(){return function(){var d=this;function f(a){return\"string\"==typeof a};function h(a,b){this.code=a;this.a=l[a]||m;this.message=b||\"\";a=this.a.replace(/((?:^|\\s+)[a-z])/g,function(a){return a.toUpperCase().replace(/^[\\s\\xa0]+/g,\"\")});b=a.length-5;if(0>b||a.indexOf(\"Error\",b)!=b)a+=\"Error\";this.name=a;a=Error(this.message);a.name=this.name;this.stack=a.stack||\"\"}\n(function(){var a=Error;function b(){}b.prototype=a.prototype;h.b=a.prototype;h.prototype=new b;h.prototype.constructor=h;h.a=function(b,c,g){for(var e=Array(arguments.length-2),k=2;k<arguments

2021-02-26 21:13:37 [urllib3.connectionpool] DEBUG: http://127.0.0.1:41779 "POST /session/1bc17f49757f2ac2366f3e111feec946/execute/sync HTTP/1.1" 200 15
2021-02-26 21:13:37 [selenium.webdriver.remote.remote_connection] DEBUG: Finished Request
2021-02-26 21:13:37 [selenium.webdriver.remote.remote_connection] DEBUG: POST http://127.0.0.1:41779/session/1bc17f49757f2ac2366f3e111feec946/element {"using": "xpath", "value": "//*[@id=\"option1\"]/option[not(@value=\"\")]"}
2021-02-26 21:13:37 [urllib3.connectionpool] DEBUG: http://127.0.0.1:41779 "POST /session/1bc17f49757f2ac2366f3e111feec946/element HTTP/1.1" 200 88
2021-02-26 21:13:37 [selenium.webdriver.remote.remote_connection] DEBUG: Finished Request
2021-02-26 21:13:37 [selenium.webdriver.remote.remote_connection] DEBUG: POST http://127.0.0.1:41779/session/1bc17f49757f2ac2366f3e111feec946/element/623db7d4-8953-4802-a8de-39469b17bb96/click {"id": "623db7d4-8953-4802-a8de-39469b17bb96"}
2021-02-26 21:13:37 [urllib3.connectionpool] DEBUG: 

In [14]:
pd.read_csv('./musinsa/datas/top_s.csv').tail()

,brand,colors,img_link,item_id,kw,link,most_age,most_sex,o_price,s_price,size_details,sizes,title
81,NaN,NaN,NaN,NaN,NaN,NaN,24~28세,남성,NaN,NaN,NaN,"S,M,L,XL,2XL",NaN
82,NaN,NaN,NaN,NaN,NaN,NaN,19~23세,남성,NaN,NaN,NaN,"S,M,L,XL",NaN
83,NaN,NaN,NaN,NaN,NaN,NaN,24~28세,남성,NaN,NaN,NaN,"M,L,XL",NaN
84,NaN,NaN,NaN,NaN,NaN,NaN,24~28세,남성,NaN,NaN,NaN,"S,M,L,XL,XXL",NaN
85,NaN,NaN,NaN,NaN,NaN,NaN,24~28세,남성,NaN,NaN,NaN,"S,M,L,XL,XXL",NaN


## argument 설정 : sid(카테고리), page(페이지)

In [32]:
%pwd

'/home/ubuntu/python3/notebook/project/crawling-repo-3'

In [53]:
%%writefile musinsa/musinsa/spiders/spider.py

import scrapy
from musinsa.items import MusinsaItem
from fake_useragent import UserAgent


class Spider(scrapy.Spider):
    name = "Musinsa"
    allow_domain = ["musinsa.com"]
    custom_settings = {
        'CONCURRENT_REQUESTS': 8,
#         'DOWNLOAD_DELAY': 2.0,
        'AUTOTHROTTLE_ENABLED': True,
        "DOWNLOADER_MIDDLEWARES": {
            "scrapy.downloadermiddlewares.useragent.UserAgentMiddleware": None,
            'scrapy.downloadermiddlewares.retry.RetryMiddleware': None,
            "scrapy_fake_useragent.middleware.RandomUserAgentMiddleware": 400,
            'scrapy_fake_useragent.middleware.RetryUserAgentMiddleware': 401,
        }
    }
    
    def __init__(self, category="001", page="1", **kwargs):
        self.start_urls = [f"https://search.musinsa.com/category/001{category}?page={page}"]
        super().__init__(**kwargs)
        
    def start_requests(self):
        for url in self.start_urls:
            yield scrapy.Request(url, callback=self.parse)
    
    def parse(self, response):
        links = response.xpath('//*[@id="searchList"]/li/div[contains(@class,"li_inner")]/div[2]/p[2]/a/@href').extract()
        for link in links:
            yield scrapy.Request(link, callback=self.parse_content)
            
    def parse_content(self, response):
        item = MusinsaItem()
        title = response.xpath('/html/head/title/text()')[0].extract().split(' -')[0]
        item["title"] = title.split(') ')[1]
        item["brand"] = response.xpath('//*[@id="product_order_info"]/div[1]/ul/li[1]/p[2]/strong/a/text()')[0].extract()
        try:
            item["o_price"] = response.xpath('//*[@id="goods_price"]/del/text()')[0].extract().strip()
        except:
            item["o_price"] = response.xpath('//*[@id="goods_price"]/text()')[0].extract().strip()
        try:
            item["s_price"] = response.xpath('//*[@id="sale_price"]/text()')[0].extract().strip()
        except:
            item["s_price"] = item["o_price"]
        kw = response.xpath('//*[@id="product_order_info"]/div[1]/ul/li[contains(@class, "article-tag-list")]/p/a/text()').extract()
        item["kw"] = list(map(lambda s : s.replace("#",''), kw))
        item["img_link"] = "https:" + response.xpath('//*[@id="detail_bigimg"]/div[1]/img/@src')[0].extract()
        item["link"] = response.url
        item["item_id"] = item["link"].split('/')[-1]
        size_kind = response.xpath('//*[@id="size_table"]/tbody/tr/th/text()').extract()
        item["size_details"] = {}
        for idx in range(len(size_kind)-1):
            sd = response.xpath(f'//*[@id="size_table"]/tbody/tr[{idx+3}]/*/text()').extract()
            item["size_details"][sd[0]] = sd[1:]

        yield item

Overwriting musinsa/musinsa/spiders/spider.py


In [49]:
%pwd

'/home/ubuntu/python3/notebook/project/crawling-repo-3'

In [54]:
%%writefile run.sh
cd ~/python3/notebook/project/crawling-repo-3/musinsa
scrapy crawl Musinsa -o ./datas/top_a.csv -a page=2

Overwriting run.sh


In [55]:
!/bin/bash run.sh

2021-02-26 19:45:56 [scrapy.utils.log] INFO: Scrapy 2.4.1 started (bot: musinsa)
2021-02-26 19:45:56 [scrapy.utils.log] INFO: Versions: lxml 4.6.2.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 20.3.0, Python 3.6.9 (default, Dec 15 2020, 04:39:26) - [GCC 7.5.0], pyOpenSSL 20.0.1 (OpenSSL 1.1.1i  8 Dec 2020), cryptography 3.3.1, Platform Linux-5.4.0-1037-aws-x86_64-with-debian-buster-sid
2021-02-26 19:45:56 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.epollreactor.EPollReactor
2021-02-26 19:45:56 [scrapy.crawler] INFO: Overridden settings:
{'AUTOTHROTTLE_ENABLED': True,
 'BOT_NAME': 'musinsa',
 'CONCURRENT_REQUESTS': 8,
 'NEWSPIDER_MODULE': 'musinsa.spiders',
 'SPIDER_MODULES': ['musinsa.spiders']}
2021-02-26 19:45:56 [scrapy.extensions.telnet] INFO: Telnet Password: 1db4029c328f0c23
2021-02-26 19:45:56 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensio

2021-02-26 19:46:09 [scrapy.core.scraper] DEBUG: Scraped from <200 https://store.musinsa.com/app/goods/1220354>
{'brand': 'CARHARTT',
 'img_link': 'https://image.msscdn.net/images/goods_img/20191111/1220354/1220354_1_500.jpg',
 'item_id': '1220354',
 'kw': ['남자반팔티', '여자반팔티', '칼하트로고티', '칼하트반팔', '칼하트반팔티', '칼하트티셔츠'],
 'link': 'https://store.musinsa.com/app/goods/1220354',
 'o_price': '79,000',
 's_price': '28,800',
 'size_details': {'2XL': ['84', '61', '74', '30'],
                  'L': ['79', '53', '61', '27.5'],
                  'M': ['78', '49.5', '57', '26'],
                  'S': ['76.5', '48', '53', '24.5'],
                  'XL': ['82', '55', '66.5', '28.5']},
 'title': 'K195 그래픽 로고 반팔티 (4컬러)'}
2021-02-26 19:46:09 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://store.musinsa.com/app/goods/808717> (referer: https://search.musinsa.com/category/001001?page=2)
2021-02-26 19:46:10 [scrapy.core.scraper] DEBUG: Scraped from <200 https://store.musinsa.com/app/goods/808717>
{'bra

2021-02-26 19:46:14 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://store.musinsa.com/app/goods/1808321> (referer: https://search.musinsa.com/category/001001?page=2)
2021-02-26 19:46:14 [scrapy.core.scraper] DEBUG: Scraped from <200 https://store.musinsa.com/app/goods/1808321>
{'brand': 'TOMMY HILFIGER',
 'img_link': 'https://image.msscdn.net/images/goods_img/20210222/1808321/1808321_1_500.jpg',
 'item_id': '1808321',
 'kw': ['코튼아이콘플래그반팔티셔츠', 'T12B1TPO460MT1', 'T12B'],
 'link': 'https://store.musinsa.com/app/goods/1808321',
 'o_price': '98,000',
 's_price': '98,000',
 'size_details': {},
 'title': '코튼 아이콘 플래그 반팔 티셔츠 (T12B1TPO460MT1YBR)'}
2021-02-26 19:46:15 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://store.musinsa.com/app/goods/1808358> (referer: https://search.musinsa.com/category/001001?page=2)
2021-02-26 19:46:15 [scrapy.core.scraper] DEBUG: Scraped from <200 https://store.musinsa.com/app/goods/1808358>
{'brand': 'TOMMY HILFIGER',
 'img_link': 'https://image.msscdn

2021-02-26 19:46:18 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://store.musinsa.com/app/goods/1814635> (referer: https://search.musinsa.com/category/001001?page=2)
2021-02-26 19:46:18 [scrapy.core.scraper] DEBUG: Scraped from <200 https://store.musinsa.com/app/goods/1814635>
{'brand': '235LABORATORY',
 'img_link': 'https://image.msscdn.net/images/goods_img/20210225/1814635/1814635_1_500.jpg',
 'item_id': '1814635',
 'kw': ['오버핏반팔', '베이직티셔츠', '루즈핏티셔츠', '라운드티', '오버핏티셔츠'],
 'link': 'https://store.musinsa.com/app/goods/1814635',
 'o_price': '29,000',
 's_price': '29,000',
 'size_details': {'FREE': ['73', '62', '64', '22']},
 'title': 'MAN 베이직 오버 무지반팔티셔츠 카키'}
2021-02-26 19:46:18 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://store.musinsa.com/app/goods/1814426> (referer: https://search.musinsa.com/category/001001?page=2)
2021-02-26 19:46:19 [scrapy.core.scraper] DEBUG: Scraped from <200 https://store.musinsa.com/app/goods/1814426>
{'brand': 'JUCYJUDY',
 'img_link': 'https:/

2021-02-26 19:46:23 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://store.musinsa.com/app/goods/1814419> (referer: https://search.musinsa.com/category/001001?page=2)
2021-02-26 19:46:23 [scrapy.core.scraper] DEBUG: Scraped from <200 https://store.musinsa.com/app/goods/1814419>
{'brand': 'JUCYJUDY',
 'img_link': 'https://image.msscdn.net/images/goods_img/20210225/1814419/1814419_1_500.jpg',
 'item_id': '1814419',
 'kw': ['반팔티', '루즈핏반팔티', '그래픽반팔티', '앞판그래픽티셔츠'],
 'link': 'https://store.musinsa.com/app/goods/1814419',
 'o_price': '69,000',
 's_price': '69,000',
 'size_details': {'080': ['63.2', '52.3', '50', '21.2'],
                  '085': ['64', '53.3', '52.5', '21.8']},
 'title': 'J앞판그래픽루즈핏티셔츠 JUTS421H'}
2021-02-26 19:46:24 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://store.musinsa.com/app/goods/1814638> (referer: https://search.musinsa.com/category/001001?page=2)
2021-02-26 19:46:24 [scrapy.core.scraper] DEBUG: Scraped from <200 https://store.musinsa.com/app/goods/181

2021-02-26 19:46:27 [scrapy.core.scraper] DEBUG: Scraped from <200 https://store.musinsa.com/app/goods/1772256>
{'brand': 'ANDERSSON BELL',
 'img_link': 'https://image.msscdn.net/images/goods_img/20210202/1772256/1772256_2_500.jpg',
 'item_id': '1772256',
 'kw': ['반팔티셔츠', '반팔', '티셔츠', '남자반팔', '반팔티'],
 'link': 'https://store.musinsa.com/app/goods/1772256',
 'o_price': '54,000',
 's_price': '54,000',
 'size_details': {'L': ['73', '54', '58', '22.5'],
                  'M': ['71', '52', '56', '21.5'],
                  'S': ['68', '49', '53', '20'],
                  'XL': ['75', '56', '60', '23.5']},
 'title': '유니섹스 스마일 어스 엠보더리 티셔츠 atb524u(BLACK)'}
2021-02-26 19:46:28 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://store.musinsa.com/app/goods/385379> (referer: https://search.musinsa.com/category/001001?page=2)
2021-02-26 19:46:28 [scrapy.core.scraper] DEBUG: Scraped from <200 https://store.musinsa.com/app/goods/385379>
{'brand': 'OVERR',
 'img_link': 'https://image.msscdn.net/imag

2021-02-26 19:46:36 [scrapy.core.scraper] DEBUG: Scraped from <200 https://store.musinsa.com/app/goods/1294379>
{'brand': 'OUTDOOR PRODUCTS',
 'img_link': 'https://image.msscdn.net/images/goods_img/20200206/1294379/1294379_1_500.jpg',
 'item_id': '1294379',
 'kw': ['로고'],
 'link': 'https://store.musinsa.com/app/goods/1294379',
 'o_price': '19,900',
 's_price': '19,900',
 'size_details': {'L': ['73.5', '51.5', '56.5', '24.5'],
                  'M': ['72', '50', '54', '23'],
                  'S': ['67', '47.5', '50', '20'],
                  'XL': ['75', '53', '59', '26']},
 'title': '스몰로고반팔티 SMALL LOGO TS'}
2021-02-26 19:46:37 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://store.musinsa.com/app/goods/1266397> (referer: https://search.musinsa.com/category/001001?page=2)
2021-02-26 19:46:37 [scrapy.core.scraper] DEBUG: Scraped from <200 https://store.musinsa.com/app/goods/1266397>
{'brand': 'FILA',
 'img_link': 'https://image.msscdn.net/images/goods_img/20200109/1266397/1266397_

2021-02-26 19:46:44 [scrapy.core.scraper] DEBUG: Scraped from <200 https://store.musinsa.com/app/goods/1452635>
{'brand': 'LEE',
 'img_link': 'https://image.msscdn.net/images/goods_img/20200519/1452635/1452635_1_500.jpg',
 'item_id': '1452635',
 'kw': ['로고', '리반팔티', '리청바지', '반팔티', '빅로고', 'LEE반팔티', 'LEE청바지'],
 'link': 'https://store.musinsa.com/app/goods/1452635',
 'o_price': '29,000',
 's_price': '14,500',
 'size_details': {'L': ['74', '53', '58', '26'],
                  'M': ['71', '51.5', '55.5', '25'],
                  'S': ['69.5', '50', '53', '24'],
                  'XL': ['76', '54.5', '60.5', '27']},
 'title': 'POCKET SMALL LOGO HALF T-SHIRT WHITE'}
2021-02-26 19:46:46 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://store.musinsa.com/app/goods/530484> (referer: https://search.musinsa.com/category/001001?page=2)
2021-02-26 19:46:47 [scrapy.core.scraper] DEBUG: Scraped from <200 https://store.musinsa.com/app/goods/530484>
{'brand': 'GILDAN',
 'img_link': 'https://image.m

2021-02-26 19:46:55 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://store.musinsa.com/app/goods/499694> (referer: https://search.musinsa.com/category/001001?page=2)
2021-02-26 19:46:56 [scrapy.core.scraper] DEBUG: Scraped from <200 https://store.musinsa.com/app/goods/499694>
{'brand': 'SLOW ACID',
 'img_link': 'https://image.msscdn.net/images/goods_img/20170315/499694/499694_1_500.jpg',
 'item_id': '499694',
 'kw': ['반팔', '반팔티', '백프린팅', '트레일러'],
 'link': 'https://store.musinsa.com/app/goods/499694',
 'o_price': '36,000',
 's_price': '28,800',
 'size_details': {'10수싱글': ['70', '46.5', '56', '25'],
                  '20수싱글': ['70', '46.5', '56', '25']},
 'title': '[원단선택가능] 트레일러 반팔티 (black)'}
2021-02-26 19:46:56 [scrapy.extensions.logstats] INFO: Crawled 82 pages (at 82 pages/min), scraped 80 items (at 80 items/min)
2021-02-26 19:46:56 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://store.musinsa.com/app/goods/996181> (referer: https://search.musinsa.com/category/001001?page

In [56]:
pd.read_csv('./musinsa/datas/top_a.csv').tail()

,brand,colors,img_link,item_id,kw,link,most_age,most_sex,o_price,s_price,size_details,sizes,title
84,MUSINSA STANDARD,NaN,https://image.msscdn.net/images/goods_img/2019...,996508,"기본티,라운드넥,루즈핏,면티셔츠,무지티,반팔티,MU-T,홈웨어컬렉션",https://store.musinsa.com/app/goods/996508,NaN,NaN,"12,900","12,900","{'S': ['69.5', '50', '53', '24'], 'M': ['71', ...",NaN,릴렉스 핏 크루 넥 포켓 반팔 티셔츠 [블랙]
85,COVERNAT,NaN,https://image.msscdn.net/images/goods_img/2019...,968976,"로고,로고티,반팔티,오버핏",https://store.musinsa.com/app/goods/968976,NaN,NaN,"39,000","39,000","{'M': ['69', '50.5', '53', '22'], 'L': ['71', ...",NaN,S/S LAYOUT LOGO TEE RED
86,SOVERMENT,NaN,https://image.msscdn.net/images/goods_img/2018...,773734,"그래픽,반팔,티,티셔츠",https://store.musinsa.com/app/goods/773734,NaN,NaN,"52,000","31,200","{'S': ['69', '54.5', '55', '23'], 'M': ['71', ...",NaN,15s invisible tee-black-
87,MARK GONZALES,NaN,https://image.msscdn.net/images/goods_img/2018...,750808,"곤잘레스,로고,반팔,반팔티셔츠,엔젤로고,오버핏반팔티,티셔츠",https://store.musinsa.com/app/goods/750808,NaN,NaN,"39,000","39,000","{'M': ['71', '52', '55', '22.5'], 'L': ['73', ...",NaN,M/G ANGEL LOGO T-SHIRTS WHITE
88,LEE,NaN,https://image.msscdn.net/images/goods_img/2020...,1473727,NaN,https://store.musinsa.com/app/goods/1473727,NaN,NaN,"29,000","14,500","{'S': ['69.5', '50', '53', '24'], 'M': ['71', ...",NaN,BOX SMALL LOGO HALF T-SHIRT WHITE/RED
